In [0]:
!pip install imdbpy

     |████████████████████████████████| 296kB 3.5MB/s 


In [0]:
import pandas as pd
import altair as alt
import numpy as np
from altair import datum

alt.data_transformers.enable('default', max_rows=None) 

DataTransformerRegistry.enable('default')

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


oscar nominants dataset
https://datahub.io/rufuspollock/oscars-nominees-and-winners

In [0]:
oscar_df = pd.read_csv('/content/drive/My Drive/OscarData.csv')
oscar_df.tail()

IMDB dataset: https://imdbpy.github.io

In [0]:
from imdb import IMDb
ia = IMDb()

## Most nominated actors/acresses

In [0]:
best_actor_actress_nominated_df = oscar_df[(oscar_df.category.isin(['ACTOR IN A LEADING ROLE', 
                                                                    'ACTOR IN A SUPPORTING ROLE', 
                                                                    'ACTOR', 
                                                                    'ACTRESS IN A LEADING ROLE', 
                                                                    'ACTRESS IN A SUPPORTING ROLE', 
                                                                    'ACTRESS']))]

nominated_count = best_actor_actress_nominated_df.entity.value_counts()
winner_count = best_actor_actress_nominated_df[best_actor_actress_nominated_df.winner==True].entity.value_counts()

best_actors_actress_count_df = pd.DataFrame(data=dict(nominated=nominated_count, 
                       win=winner_count, name = nominated_count.index), 
             index=nominated_count.index)

best_actors_actress_count_df.head()

In [0]:
best_actors_for_viz_df = pd.DataFrame(columns=['nominated', 'name', 'win'])

for ind, row in best_actors_actress_count_df[best_actors_actress_count_df.nominated>6].iterrows():
    if row['nominated'] > 1:
        for i in range(1, row['nominated']+1):
            best_actors_for_viz_df = best_actors_for_viz_df.append({'nominated': i, 'name': row['name']}, ignore_index=True)
    if row['win'] > 0:
        for i in range(int(row['win'])):
            best_actors_for_viz_df = best_actors_for_viz_df.append({'win': row['nominated']-i, 'name': row['name']}, ignore_index=True)
    

## Best pictires

In [0]:
oscar_imdb_df = oscar_df[(oscar_df.category == 'BEST PICTURE')  & 
                                 (oscar_df.winner == True) & (oscar_df.year > 1989)][['entity', 'year']]

In [0]:
oscar_imdb_df['year'] = oscar_imdb_df['year']+1
oscar_imdb_df['budget'] = ''
oscar_imdb_df['box_office'] = ''
oscar_imdb_df['rating'] = ''

for ind, row in oscar_imdb_df.iterrows():
    movie = ia.search_movie(row['entity'])[0]
    ia.update(movie, info=['main'])
    oscar_imdb_df.loc[ind, 'budget'] = movie['box office']['Budget']if 'box office' in movie and 'Budget' in movie['box office'] else None
    oscar_imdb_df.loc[ind, 'box_office'] = movie['box office']['Cumulative Worldwide Gross'] if 'box office' in movie and 'Cumulative Worldwide Gross' in movie['box office'] else None
    oscar_imdb_df.loc[ind, 'rating'] = movie['rating'] 

In [0]:
import re
oscar_imdb_df.fillna(0, inplace=True)

extract_currency = lambda s: re.findall("\$([-0-9.,]+[-0-9.,]*)", str(s))[0].replace(',', '') if re.findall("\$([-0-9.,]+[-0-9.,]*)", str(s)) else 0

oscar_imdb_df.budget = oscar_imdb_df.budget.apply(extract_currency)
oscar_imdb_df.budget = oscar_imdb_df.budget.astype(int)

oscar_imdb_df.box_office = oscar_imdb_df.box_office.apply(extract_currency)
oscar_imdb_df.box_office = oscar_imdb_df.box_office.astype(int)

## Directors

In [0]:
best_directed_films_df = oscar_df[(oscar_df.category == 'DIRECTING')].drop(columns=['category']).reset_index()
best_directed_films_df['director'] = ''

In [0]:
for ind, row in best_directed_films_df.iterrows():
    movie = ia.search_movie(row['entity'])
    if movie:
      movie=movie[0]
      ia.update(movie, info=['main'])
      best_directed_films_df.loc[ind,'director'] = movie['director'][0]['name'] if 'director' in movie else None


In [0]:
best_directed_films_df.dropna(inplace=True)

In [0]:
dir_nominated_count = best_directed_films_df.director.value_counts()
dir_winner_count = best_directed_films_df[best_directed_films_df.winner==True].director.value_counts()

best_dir_count_df = pd.DataFrame(data=dict(nominated=dir_nominated_count, 
                       win=dir_winner_count, director = dir_nominated_count.index), 
             index=dir_nominated_count.index)

best_dir_count_df.head()

,nominated,win,director
William Wyler,9,3.0,William Wyler
Billy Wilder,8,2.0,Billy Wilder
Martin Scorsese,8,1.0,Martin Scorsese
Fred Zinnemann,7,2.0,Fred Zinnemann
Woody Allen,7,1.0,Woody Allen


## Visualizations

In [0]:
background_color = '#f5f5f5'
med_grey = '#9d9d9d'
orange = '#fdbf11'
font = 'Open Sans'
title_size= 15
title_color = 'grey'

labels_color = 'grey'
labels_size = 12

paddings = {'top': 40, 'left': 40, 'right': 40, 'bottom':40}

In [0]:
oscar_imdb_df.replace('The Lord of the Rings: The Return of the King', 'The Lord of the Rings 3', inplace = True)
oscar_imdb_df.replace('Birdman or (The Unexpected Virtue of Ignorance)', 'Birdman', inplace = True)

sorted_movies = list(oscar_imdb_df.sort_values(['year'], ascending=False).index)

base = alt.Chart(oscar_imdb_df, width=600, height=800)

chart = alt.layer(
    base.mark_bar(color='black', height=10).encode(
    y=alt.Y('year:N', sort = alt.Sort(field='year', order= 'descending')),
    x=alt.X('box_office:Q', axis = alt.Axis(), scale=alt.Scale(type='log')),
    ),
  base.mark_bar(color='red', height=10).encode(
    y=alt.Y('year:N'),
    x=alt.X('budget:Q', scale=alt.Scale(type='log'))
  ),
    base.mark_circle(opacity=1, color='orange').encode(
    y=alt.Y('year:N'),
    x=alt.X('box_office:Q', scale=alt.Scale(type='log'), axis = alt.Axis(values = [1000000, 3000000, 10000000, 30000000, 100000000, 300000000, 1000000000, 3000000000],format="$~s")),
        size = alt.Size('rating:Q', scale= alt.Scale(range=[150, 800], domain=[7, 9]), legend=alt.Legend(direction='horizontal', legendY=30, legendX=225, orient= 'none', title=None)),
        tooltip = alt.Tooltip(['entity:N', 'year:N', 'box_office:Q', 'budget:Q', 'rating:N'])
  ),
  base.mark_text(align='left', font = font, dx=-520, size = labels_size, color=labels_color).encode(y=alt.Y('year:N'), text = 'entity:N')
)

title = alt.Chart(
    {"values": [{"text": "Oscar: The Best Pictures from 1991 to 2018"}]}
).mark_text(size=30, align='left', font = font, color = 'grey', dx=-120, dy = -10).encode(
    text="text:N"
)

legend = alt.layer(
    alt.Chart({'values': [{"x1":0, "x2":40, 'text': 'budget'}]}).mark_rect(color='red', height=10, dy=10).encode(x = alt.X('x1:Q', axis=alt.Axis(grid=False, labels=False)), x2='x2:Q', ),
    alt.Chart({'values': [{"x1":80, "x2":120, 'text': 'box office'}]}).mark_rect(color='black', height=10).encode(x = alt.X('x1:Q', axis=alt.Axis(grid=False, labels=False), scale=alt.Scale(range=[-50, 470])), x2='x2:Q', ),
    alt.Chart({'values': [{"x":40, 'text': 'budget'}, {"x":120, 'text': 'box office'}, {"x":350, 'text':'IMDB rating'}]}).mark_text(color = 'grey', align='left', dx = 10, dy=-10, font = font).encode(x = 'x:Q', text='text:N'),
    )

best_pics = (title & legend & chart).properties(
    background = background_color,
    padding= paddings,
).configure_axis(
    title = None,
        ticks=False, 
        domain=False,
        labelPadding=10,
        labelFont= font,
        labelFontSize=labels_size,
        labelColor=labels_color,       
).configure_view(
    stroke=None,
).configure_title(
    font=font,
)
# best_pics

In [0]:
sort_actors = list(best_actors_actress_count_df[best_actors_actress_count_df.nominated>6].sort_values(['nominated', 'win'], ascending=False).index)

base = alt.Chart(best_actors_for_viz_df, width= 600, height=800)

chart = alt.layer(
  base.mark_circle(color=med_grey, size=550, opacity=0.9).encode(
    x = alt.X('nominated:Q', axis=alt.Axis(labels=False, title=None)),
    y = alt.Y('name:N', sort = alt.SortArray(sort_actors), axis=alt.Axis(title=None))),
      
  base.mark_circle(color=orange, size=400, opacity=1).encode(
    x = alt.X('win:Q'),
    y = alt.Y('name:N', sort = alt.SortArray(sort_actors),
              axis=alt.Axis(title=None))),
          

  base.mark_text(color='white', font=font).encode(
    x = alt.X('max_win:Q'),
    y = alt.Y('name:N', sort = alt.SortArray(sort_actors), axis=alt.Axis(title=None)),
    text = 'win_count:Q')
    .transform_aggregate(
      max_win='max(win)',
      min_win='min(win)',
      groupby=["name"]).transform_calculate(
        win_count= datum.max_win - datum.min_win + 1,
      ),

  base.mark_text(color='black', font=font).encode(
      x = alt.X('nom_label:Q'),
      y = alt.Y('name:N', sort = alt.SortArray(sort_actors),
              axis=alt.Axis(title=None)),
      text = 'nom_count:Q')
      .transform_aggregate(
      max_nom='max(nominated)',
      min_nom='min(nominated)',
      groupby=["name"]).transform_calculate(
          nom_count = datum.max_nom - datum.min_nom + 1,
        nom_label = datum.max_nom - datum.min_nom + 1 + 1
      ), 
)

title = alt.Chart(
    {"values": [{"text": "Oscar: The Most nominated actors"}]}
).mark_text(size=30, align='left', color = 'grey', dx=15, dy = -10, font=font).encode(
    text="text:N"
)

legend = alt.layer(
    alt.Chart({'values': [{"x":2, 'text': 'nominated'}, {"x":6, 'text': 'winner'}]}).mark_circle(size=550, color='grey').encode(x = 'x:Q', color=alt.Color('text:N', scale = alt.Scale(range=['grey', 'orange']), legend=None)),
    alt.Chart({'values': [{"x":2, 'text': 'nominated'}, {"x":6, 'text': 'winner'}]}).mark_text(color = 'grey', align='left', dx = 15, font=font).encode(x = 'x:Q', text='text:N'),)

most_nom_actors = (title & legend & chart).properties(
    background = background_color,
    padding= paddings,
).configure_axisX(
    title=None,
    labels=False,
).configure_axis(
        grid= False, 
        ticks=False, 
        domain=False,
        labelFont= font,
        labelColor=labels_color,
        labelFontSize=labels_size,
).configure_view(
    stroke=None,
)


In [0]:
from altair import expr, datum

_sort = list(best_dir_count_df[best_dir_count_df.nominated>3].sort_values(['nominated', 'win'], ascending=False).index)

base = alt.Chart(best_dir_count_df[best_dir_count_df.nominated>3], height=800, width=600)

chart = alt.layer(
    base.mark_bar(color='grey', height=18).encode(
        x = alt.X('nominated:Q',),
        y = alt.Y('director:N', sort = alt.SortArray(_sort),),
        
    ),
    
    base.mark_text(color = 'white', align='right', baseline='middle', dx = -3, font=font).encode(
        x = alt.X('nominated:Q'),
        y = alt.Y('director:N', sort = alt.SortArray(_sort)),
        text='nominated:Q'
    ),
    base.mark_text(color = 'black', opacity=1, font=font).encode(
        x = alt.X('x:Q'),
        y = alt.Y('director:N', sort = alt.SortArray(_sort)),
        text='win:Q'
    ).transform_calculate(
      x=datum.nominated + 0.4,
    ).transform_filter(
    'datum.win > 0'
),
    base.mark_circle(color='orange', align='right', baseline='middle', size=650).encode(
        x = alt.X('x:Q'),
        y = alt.Y('director:N', sort = alt.SortArray(_sort)),
        size=alt.Size('win:Q', scale=alt.Scale(range=[900, 1400], domain=[1,3]), legend=None)
    ).transform_calculate(
      x=datum.nominated + 0.4,
    )
)

title = alt.Chart(
    {"values": [{"text": "Oscar: The Most Nominated Directors"}]}
).mark_text(size=30, align='left', font = font, color = 'grey', dx=-25, dy = -10).encode(
    text="text:N"
)

legend = alt.layer(
    alt.Chart({'values': [{"x":2.5, 'text': 'winner'}]}).mark_circle(size=550, color='orange').encode(x = 'x:Q'),
    alt.Chart({'values': [{"x1":1.3, "x2":0.3, 'text': 'nominated'}]}).mark_rect(color='grey').encode(x = 'x1:Q', x2='x2:Q'),
    alt.Chart({'values': [{"x":1.25, 'text': 'nominated'}, {"x":2.5, 'text': 'winner'}]}).mark_text(color = 'grey', align='left', dx = 15, font=font).encode(x = 'x:Q', text='text:N'),)


most_nom_directors = (title & legend & chart).properties(
    background = background_color,
    # title = 'Oscar: The Most Nominated Directors',
    padding=paddings,
).configure_axis(
        title=None,
        grid= False, 
        ticks=False, 
        domain=False,
        labelPadding=10,
        labelFont= font,
        labelColor=labels_color,
        labelFontSize=labels_size,
).configure_axisX(
    labels=False,
).configure_view(
    stroke=None,
).configure_title(
    fontSize=title_size,
    font=font,
    color=title_color,
).configure_legend(title=None)



In [0]:
most_nom_actors

alt.VConcatChart(...)

In [0]:
most_nom_directors

alt.VConcatChart(...)

In [0]:
best_pics

alt.VConcatChart(...)